In [1]:
import sys
import cv2
from cvzone.PoseModule import PoseDetector
import numpy as np


# source image
imgPath = './images/template_01/'
img_A = cv2.imread(imgPath + 'A.png')
img_B = cv2.imread(imgPath + 'B.png')
img_out = cv2.imread(imgPath + 'A.png')
H = img_A.shape[0]
W = img_A.shape[1]
print(H, W)
if img_B.shape[0] != H or img_B.shape[1] != W:
    sys.exit(" QAQ Error: two images have different sizes")   

    
# temptlate (offset, size) (y,x)
tplt = np.array([[600,   420,  472, 2530],  # block0 : Head
                 [1110,  420, 2340, 4100],  # block1 : Pic
                 [3480,  420,  820, 4520],  # block2 : Paragraph
                 [140,  5300,  800, 2500],  # block3 : Weather
                 [1240, 5300, 1250, 2490],  # block4 : Edit fav
                 [2890, 5300, 1500, 2490]]) # block5 : Comment


# resizing
reduceFactor = 4
img_A = cv2.resize(img_A, None, fx=(1/reduceFactor), fy=(1/reduceFactor))
img_B = cv2.resize(img_B, None, fx=(1/reduceFactor), fy=(1/reduceFactor))
img_out = cv2.resize(img_out, None, fx=(1/reduceFactor), fy=(1/reduceFactor))
tplt = tplt//reduceFactor


# source segment
img_segment_0A = img_A[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :]
img_segment_0B = img_B[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :]
img_segment_1A = img_A[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :]
img_segment_1B = img_B[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :]
img_segment_2A = img_A[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :]
img_segment_2B = img_B[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :]
img_segment_3A = img_A[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :]
img_segment_3B = img_B[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :]
img_segment_4A = img_A[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :]
img_segment_4B = img_B[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :]
img_segment_5A = img_A[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :]
img_segment_5B = img_B[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :]


# status variable & parameter
blockState = [0,0,0,0,0,0]
blockTrigger = [False, False, False, False, False, False]
duration = 20


# detector & capture video
detector = PoseDetector()
cap = cv2.VideoCapture(1)
cap_height = 540 # size = 1/2 => shape: (540, 960, 3)
cap_width = 960
print("cap.isOpened:", cap.isOpened())


while True:
    
    
    # 
    # Detecting: update blockTrigger[] depending on the cvzone result
    #

    success, frame = cap.read()
    if not success:
        print("cap.read failed")
        break
    #frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, None, fx=(1/2), fy=(1/2))
    detector.findPose(frame, draw=True)
    lmlist, bbox = detector.findPosition(frame, draw=True)
    
    if bbox:
        center = bbox["center"]
        if center[0] < 0 or center[0] > cap_width or center[1] < 0 or center[1] > cap_height:
            #print("ERROR: center[] outside the window", center)
            pass
        
        # 0 
        elif center[0] < cap_width // 3 and center[1] < cap_height // 3:
            cv2.putText(frame, "Block 0", (100, 100), cv2.FONT_HERSHEY_PLAIN, 3, (0, 150, 255), 2)
            if blockState[0] % duration == 0:
                # block is static
                blockTrigger[0] = True
            
        # 1
        elif center[0] < (cap_width // 3) * 2 and center[1] < cap_height // 3:
            cv2.putText(frame, "Block 1", (100, 100), cv2.FONT_HERSHEY_PLAIN, 3, (0, 150, 255), 2)
            if blockState[1] % duration == 0:
                blockTrigger[1] = True

        # 2
        elif True and center[1] < cap_height // 3:
            cv2.putText(frame, "Block 2", (100, 100), cv2.FONT_HERSHEY_PLAIN, 3, (0, 150, 255), 2)
            if blockState[2] % duration == 0:
                blockTrigger[2] = True
            
        # 3
        elif center[0] < cap_width // 3:
            cv2.putText(frame, "Block 3", (100, 100), cv2.FONT_HERSHEY_PLAIN, 3, (0, 150, 255), 2)
            if blockState[3] % duration == 0:
                blockTrigger[3] = True
            
        # 4
        elif center[0] < (cap_width // 3) * 2:
            cv2.putText(frame, "Block 4", (100, 100), cv2.FONT_HERSHEY_PLAIN, 3, (0, 150, 255), 2)
            if blockState[4] % duration == 0:
                blockTrigger[4] = True

        # 5
        else:
            cv2.putText(frame, "Block 5", (100, 100), cv2.FONT_HERSHEY_PLAIN, 3, (0, 150, 255), 2)
            if blockState[5] % duration == 0:
                blockTrigger[5] = True
        
        cv2.putText(frame, str(center), (100, 500), cv2.FONT_HERSHEY_PLAIN, 2, (0, 150, 255), 2)
        cv2.putText(frame, str(blockTrigger), (100, 300), cv2.FONT_HERSHEY_PLAIN, 2, (0, 150, 255), 2)
        cv2.putText(frame, str(blockState), (100, 400), cv2.FONT_HERSHEY_PLAIN, 2, (0, 150, 255), 2)
        
    # end if
    
    #
    # end of Detecting
    #
    
    #################################
        
    #   
    # Render according to blockState[]
    #    

    
    # block 0
    #ct = blockState[0]
    if blockState[0] == 0:
        img_out[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :] = img_segment_0A
        if blockTrigger[0] == True:
            blockState[0] += 1
            blockTrigger[0] = False
            
    elif blockState[0] == duration:
        img_out[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :] = img_segment_0B
        if blockTrigger[0] == True:
            blockState[0] += 1
            blockTrigger[0] = False
            
    elif blockState[0] < duration: # render A->B
        #print("A->B")
        alpha = blockState[0] / duration
        img_out[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :] = cv2.addWeighted(img_segment_0B, alpha, img_segment_0A, (1-alpha), 0)
        blockState[0] += 1
        if blockState[0] == duration:       
            print("finish A->B (block 0)")
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[0]-duration) / duration
        img_out[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :] = cv2.addWeighted(img_segment_0A, alpha, img_segment_0B, (1-alpha), 0)
        blockState[0] += 1
        if blockState[0] == (duration*2):
            print("finish B->A (block 0)")
 
    
    blockState[0] = blockState[0] % (duration * 2)
    ###
    
    
    # block1   
    if blockState[1] == 0:
        img_out[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :] = img_segment_1A
        if blockTrigger[1] == True:
            blockState[1] += 1
            blockTrigger[1] = False
            
    elif blockState[1] == duration:
        img_out[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :] = img_segment_1B
        if blockTrigger[1] == True:
            blockState[1] += 1
            blockTrigger[1] = False
            
    elif blockState[1] < duration: # render A->B
        #print("A->B")
        alpha = blockState[1] / duration
        img_out[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :] = cv2.addWeighted(img_segment_1B, alpha, img_segment_1A, (1-alpha), 0)
        blockState[1] += 1
        if blockState[1] == duration:
            print("finish A->B (block 1)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[1]-duration) / duration
        img_out[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :] = cv2.addWeighted(img_segment_1A, alpha, img_segment_1B, (1-alpha), 0)
        blockState[1] += 1
        if blockState[1] == (duration*2):
            print("finish B->A (block 1)")
            #break 
    
    blockState[1] = blockState[1] % (duration * 2) 
    ###  
    
    # block2
    if blockState[2] == 0:
        img_out[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :] = img_segment_2A
        if blockTrigger[2] == True:
            blockState[2] += 1
            blockTrigger[2] = False
            
    elif blockState[2] == duration:
        img_out[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :] = img_segment_2B
        if blockTrigger[2] == True:
            blockState[2] += 1
            blockTrigger[2] = False
            
    elif blockState[2] < duration: # render A->B
        #print("A->B")
        alpha = blockState[2] / duration
        img_out[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :] = cv2.addWeighted(img_segment_2B, alpha, img_segment_2A, (1-alpha), 0)
        blockState[2] += 1
        if blockState[2] == duration:
            print("finish A->B (block 2)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[2]-duration) / duration
        img_out[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :] = cv2.addWeighted(img_segment_2A, alpha, img_segment_2B, (1-alpha), 0)
        blockState[2] += 1
        if blockState[2] == (duration*2):
            print("finish B->A (block 2)")
            #break 
    
    blockState[2] = blockState[2] % (duration * 2) 
    ###



    # block3
    if blockState[3] == 0:
        img_out[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :] = img_segment_3A
        if blockTrigger[3] == True:
            blockState[3] += 1
            blockTrigger[3] = False
            
    elif blockState[3] == duration:
        img_out[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :] = img_segment_3B
        if blockTrigger[3] == True:
            blockState[3] += 1
            blockTrigger[3] = False
            
    elif blockState[3] < duration: # render A->B
        #print("A->B")
        alpha = blockState[3] / duration
        img_out[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :] = cv2.addWeighted(img_segment_3B, alpha, img_segment_3A, (1-alpha), 0)
        blockState[3] += 1
        if blockState[3] == duration:
            print("finish A->B (block 3)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[3]-duration) / duration
        img_out[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :] = cv2.addWeighted(img_segment_3A, alpha, img_segment_3B, (1-alpha), 0)
        blockState[3] += 1
        if blockState[3] == (duration*2):
            print("finish B->A (block 3)")
            #break 
    
    blockState[3] = blockState[3] % (duration * 2) 
    ###




    # block4
    if blockState[4] == 0:
        img_out[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :] = img_segment_4A
        if blockTrigger[4] == True:
            blockState[4] += 1
            blockTrigger[4] = False
            
    elif blockState[4] == duration:
        img_out[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :] = img_segment_4B
        if blockTrigger[4] == True:
            blockState[4] += 1
            blockTrigger[4] = False
            
    elif blockState[4] < duration: # render A->B
        #print("A->B")
        alpha = blockState[4] / duration
        img_out[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :] = cv2.addWeighted(img_segment_4B, alpha, img_segment_4A, (1-alpha), 0)
        blockState[4] += 1
        if blockState[4] == duration:
            print("finish A->B (block 4)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[4]-duration) / duration
        img_out[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :] = cv2.addWeighted(img_segment_4A, alpha, img_segment_4B, (1-alpha), 0)
        blockState[4] += 1
        if blockState[4] == (duration*2):
            print("finish B->A (block 4)")
            #break 
    
    blockState[4] = blockState[4] % (duration * 2) 
    ###
    
    
    
    # block5
    if blockState[5] == 0:
        img_out[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :] = img_segment_5A
        if blockTrigger[5] == True:
            blockState[5] += 1
            blockTrigger[5] = False
            
    elif blockState[5] == duration:
        img_out[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :] = img_segment_5B
        if blockTrigger[5] == True:
            blockState[5] += 1
            blockTrigger[5] = False
            
    elif blockState[5] < duration: # render A->B
        #print("A->B")
        alpha = blockState[5] / duration
        img_out[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :] = cv2.addWeighted(img_segment_5B, alpha, img_segment_5A, (1-alpha), 0)
        blockState[5] += 1
        if blockState[5] == duration:
            print("finish A->B (block 5)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[5]-duration) / duration
        img_out[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :] = cv2.addWeighted(img_segment_5A, alpha, img_segment_5B, (1-alpha), 0)
        blockState[5] += 1
        if blockState[5] == (duration*2):
            print("finish B->A (block 5)")
            #break 
    
    blockState[5] = blockState[5] % (duration * 2) 
    ###    


    #
    # end of Rendering
    #
    
    
    # display
    cv2.imshow("WebCam Frame", frame)
    cv2.imshow('News',img_out)
    cv2.waitKey(1)
    
    
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'): # 27=ESC 113='q'
        print("break", key)
        break
    
    
### end of main 'while loop'
    
    
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)



4500 8000
cap.isOpened: True
finish B->A (block 1)
finish A->B (block 5)
finish A->B (block 4)
finish A->B (block 3)
finish B->A (block 5)
finish B->A (block 4)
finish A->B (block 5)
finish A->B (block 2)
finish B->A (block 2)
finish A->B (block 2)
finish A->B (block 1)
finish A->B (block 4)
finish B->A (block 5)
finish A->B (block 5)
finish B->A (block 5)
finish B->A (block 4)
finish A->B (block 4)
finish B->A (block 3)
finish B->A (block 4)
finish A->B (block 5)
finish B->A (block 2)
finish B->A (block 1)
finish A->B (block 4)
finish A->B (block 0)
break 113


-1